In [17]:
from utils.consts import *

In [2]:
from utils.utilities import test, show_dep, get_data, get_nominal_subjects, get_prep, get_objects

In [3]:
import json
import random
import json
from IPython.display import Markdown, display
from pprint import pprint

In [4]:
import spacy

# nlp_pipe = spacy.load('en_core_web_lg')
PIPE = spacy.load('en_coref_lg')

In [5]:
dataset = './data/dev_dataset.json'

In [6]:
data = get_data(dataset)

In [7]:
pprint(data[0])

{'header': ['Player',
            'No.',
            'Nationality',
            'Position',
            'Years in Toronto',
            'School/Club Team'],
 'phase': 1,
 'question': 'What position does the player who played for butler cc (ks) '
             'play?',
 'sql': {'agg': '',
         'conds': [['School/Club Team', '=', 'Butler CC (KS)']],
         'sel': 'Position'},
 'table_id': '1-10015132-11'}


In [8]:
show_dep(PIPE(data[0]['question']))

In [9]:
pprint(data[1])
show_dep(PIPE(data[1]['question']))

{'header': ['Player',
            'No.',
            'Nationality',
            'Position',
            'Years in Toronto',
            'School/Club Team'],
 'phase': 1,
 'question': 'How many schools did player number 3 play at?',
 'sql': {'agg': 'COUNT',
         'conds': [['No.', '=', '3']],
         'sel': 'School/Club Team'},
 'table_id': '1-10015132-11'}


In [10]:
# get the root
def get_root(doc):
    # There is always a single root for a sentence.
    # This is just a safety guard
    roots = list()
    for word in doc:
        if word.head == word:
            roots.append(word)
    if len(roots) != 1:
        return None
    return roots[0]

In [11]:
VERB

['VERB']

In [21]:
def _is_verb(root):
    if root.pos_ in VERB:
        return True
    else:
        return False
    
def get_aux(root):
    aux_list = []
    for child in root.children:
        if child.pos_ in VERB and child.dep_ in AUX:
            aux_list.append(child)
    return aux_list

In [19]:
CASUAL_SUBJECT = ["csubj", "csubjpass"]

In [20]:
def check_if_wh(nominals):
    subj = nominals[0]
    if subj.tag_ in WH:
        return True
    return False

def get_csubj(root_verb):
    for child in root_verb.children:
        if child.dep_ in CASUAL_SUBJECT and child.pos_ in VERB:
            return child
    return None    

In [39]:
def get_verb_and_subj_pair(data_sample):
#     print("\n========================================================\n")
#     pprint(data_sample)
    print("\n========================================================\n")
    # get the doc representation of the string
    doc = PIPE(data_sample['question'])
    # Find out the root of the given statement
    root = get_root(doc)
#     import pdb; pdb.set_trace()
    is_verb = _is_verb(root)
    aux = None
    if not is_verb:
        # Some times the root can be a noun, in that case it'll always be associated to
        # an aux verb, and we gotta find it
        aux = get_aux(root)
    # If there is any aux found, we use it as our verb root
    if aux is not None and len(aux) > 0:
        root_verb = aux[0]
    else:
        # Else, we use our root as it is
        root_verb = root
    # Check if there is any csubj attached this means we have two wh nouns attached to
    # two different verbs one is asking and other qualifying.
    csubj_verb = get_csubj(root_verb)
    if csubj_verb is not None:
        root_verb = csubj_verb
#     print("ROOT : {}".format(root.text))
#     print("AUX : ")
#     pprint(aux)
    # nominals in this case is going to be a list of all the cols
    # in SELECT clause. If we find any nsubj attached to the root_verb we catch it here.
    nominals = get_nominal_subjects(root_verb)
    is_wh = False
    # If what we have found is not actually a noun but a question noun (wh-word)
    # TODO: In this case we may want to go on hunt for the actual nominal col.
    if len(nominals) != 0:
        is_wh = check_if_wh(nominals)
    if is_wh:
        # for now
        pass
#     print("SUBJ : ")
#     pprint(nominals)
    # Even after all this if there is no way the prog have found a nominal
    # find out if there is any prop attached to the verb.
    # If there is then we may look for prep_obj attached to it for the nominal.
    prep = None
    if len(nominals) == 0:
        prep = get_prep(root_verb)
#     print("PREP : ")
#     pprint(prep)
    pobj = None
    if prep is not None:
        pobj = get_objects(prep)
        nominals.append(pobj)
#     print("POBJ : ")
#     pprint(pobj)
#     show_dep(doc)
    return nominals

In [40]:
def get_subtree_list(tok):
    return [t for t in tok.subtree]

# We need to get a list of atmost 7 toks where
def get_desired_subtree(tok):
    subtree = get_subtree_list(tok)
    if len(subtree) > 7:
        id = 0
        for indx in range(len(subtree)):
            if subtree[indx] == tok:
                id = indx
                break
        lw = 0
        hs = len(subtree)
        if id - 3 > lw:
            lw = id - 3
        if id + 3 < hs:
            hs = id + 3
        subtree = subtree[lw:hs]
    return subtree        

In [41]:
data[22]

{'phase': 1,
 'table_id': '1-10342194-3',
 'question': 'What is the amount of trees, that require replacement when prevailing types, % is pine — 29.37 poplar — 26.12 acer negundo — 13.2?',
 'sql': {'sel': 'Amount of trees, that require replacement',
  'conds': [['Prevailing types, %',
    '=',
    'Pine — 29.37 Poplar — 26.12 Acer negundo — 13.2']],
  'agg': ''},
 'header': ['District',
  'Total amount of trees',
  'Prevailing types, %',
  'Amount of old trees',
  'Amount of trees, that require replacement']}

In [42]:
for idx in range(21, 30):
    subjs = get_verb_and_subj_pair(data[idx])
    if len(subjs) != 0:
        print("\n========================================================\n")
        print("SUBJs : ")
        pprint(subjs)
        print("SELECT : ")
        pprint(data[idx]['sql']['sel'])
        print("AGG INPUT : ")
        pprint(get_desired_subtree(subjs[0]))
#         pprint(data[idx])

AttributeError: 'NoneType' object has no attribute 'pos_'

In [107]:
show_dep(PIPE("What is the English name of the country whose official native language is Dutch Papiamento?"))

Find csubj for the verb
[displacy link](https://explosion.ai/demos/displacy?text=What%20is%20the%20English%20name%20of%20the%20country%20whose%20official%20native%20language%20is%20Dutch%20Papiamento%3F&model=en_core_web_lg&cpu=1&cph=0)
[coref](https://huggingface.co/coref/?text=What%20is%20the%20English%20name%20of%20the%20country%20whose%20official%20native%20language%20is%20Dutch%20Papiamento%3F)
[dep list](https://spacy.io/api/annotation#section-dependency-parsing)
